In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os
import time
import tensorflow as tf
from openpyxl import Workbook
from sklearn.metrics import *
from pandas import DataFrame, Series 
from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
from tensorflow.keras import optimizers
from tensorflow.python.client import device_lib





In [3]:
print(tf.__version__)

tf.config.list_physical_devices('GPU')


2.3.1


[PhysicalDevice(name=&#39;/physical_device:GPU:0&#39;, device_type=&#39;GPU&#39;)]

In [4]:
# 주식 데이터 가져오기
f_dir = os.getcwd()

# final_stock_predict 파일과 csv파일이 동일 디렉토리에 있을경우 불러옴
if os.path.isfile(f_dir + '/stocks.csv') and os.path.isfile(f_dir + '/trade_train.csv'):
    print("csv file check complete")
    stock_data = pd.read_csv(f_dir + '/stocks.csv')
    trade_data = pd.read_csv(f_dir + '/trade_train.csv')
    print("csv file load complete")
else :
    print("missing files")



csv file check complete
csv file load complete


In [5]:
# 타겟 종목 가져오기 
target_stock_pool = stock_data.loc[stock_data['20년7월TOP3대상여부'].str.contains('Y', na = False)]
target_stock_pool = target_stock_pool.drop_duplicates(subset = ['종목명'])
Answer  = target_stock_pool['종목명'].values.tolist()      #135개의 종목명 
Answer_code = target_stock_pool['종목번호'].values.tolist() #135개의 종목코드 

In [6]:
# 주식 거래 데이터 그룹별로 구분, grouper[n] 은 n 그룹의 거래내역
group_trade = trade_data.groupby(trade_data['그룹번호'])
group_trade = trade_data.sort_values(by = ['그룹번호'], axis = 0) 
grouper = [g[1] for g in group_trade.groupby(['그룹번호'])] 
for i in range(48):
    grouper[i] = grouper[i].sort_values(by = ['기준년월'], axis = 0)


In [7]:
# 전체 거래 종목 수 확인 및 종목코드 어레이로 변경, 1066개의 종목
all_stock = trade_data.drop_duplicates(subset = ['종목번호'])
all_stock['종목번호'].value_counts() 
Y_class = all_stock['종목번호'].values.tolist() 
Y_class = np.array(Y_class).reshape(-1,1)
len(Y_class) 

1066

In [8]:
# 전체 종목 one-hot-encoding
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder() 
enc.fit(Y_class)
Y_class_encoding = enc.transform(Y_class).toarray() 
Y_class_encoding

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
# 종목코드를 넣으면 종목명 출력 
def find_stock_name(code):
    if(code == 'A204990'):
        return '상폐'
    return stock_data.loc[stock_data['종목번호'].str.contains(code, na = False)].iloc[0]['종목명']

# 모든 주식의 코드와 종목명 따옴
all_stock_data = all_stock['종목번호'].values.tolist()
all_stock_name = []
for i in range(len(all_stock_data)):
    all_stock_name.append(find_stock_name(all_stock_data[i]))
    print(i,"는 완료\n")


0 는 완료

1 는 완료

2 는 완료

3 는 완료

4 는 완료

5 는 완료

6 는 완료

7 는 완료

8 는 완료

9 는 완료

10 는 완료

11 는 완료

12 는 완료

13 는 완료

14 는 완료

15 는 완료

16 는 완료

17 는 완료

18 는 완료

19 는 완료

20 는 완료

21 는 완료

22 는 완료

23 는 완료

24 는 완료

25 는 완료

26 는 완료

27 는 완료

28 는 완료

29 는 완료

30 는 완료

31 는 완료

32 는 완료

33 는 완료

34 는 완료

35 는 완료

36 는 완료

37 는 완료

38 는 완료

39 는 완료

40 는 완료

41 는 완료

42 는 완료

43 는 완료

44 는 완료

45 는 완료

46 는 완료

47 는 완료

48 는 완료

49 는 완료

50 는 완료

51 는 완료

52 는 완료

53 는 완료

54 는 완료

55 는 완료

56 는 완료

57 는 완료

58 는 완료

59 는 완료

60 는 완료

61 는 완료

62 는 완료

63 는 완료

64 는 완료

65 는 완료

66 는 완료

67 는 완료

68 는 완료

69 는 완료

70 는 완료

71 는 완료

72 는 완료

73 는 완료

74 는 완료

75 는 완료

76 는 완료

77 는 완료

78 는 완료

79 는 완료

80 는 완료

81 는 완료

82 는 완료

83 는 완료

84 는 완료

85 는 완료

86 는 완료

87 는 완료

88 는 완료

89 는 완료

90 는 완료

91 는 완료

92 는 완료

93 는 완료

94 는 완료

95 는 완료

96 는 완료

97 는 완료

98 는 완료

99 는 완료

100 는 완료

101 는 완료

102 는 완료

103 는 완료

104 는 완료

105 는 완료

106 는 완료

107 는 완료

108 는 완료

109 는 완료

110 는 완료



In [10]:
df = DataFrame({'종목명':all_stock_name, '인코딩순':range(len(all_stock_data))}, index = all_stock_data)

# 인코딩 함수
Answer_values = []
for i in range(len(all_stock_data)):
    Answer_values.append(i)
Answer_len = len(Answer_values)
encoding = np.eye(Answer_len)[Answer_values]

# 코드 리스트를 넣으면 인코딩된 값 리스트를 배출
def encoding_code(lst):
    encoding_final = []
    for i in range(len(lst)):
        encoding_final.append(encoding[df.loc[lst[i]]['인코딩순']].tolist())
    return encoding_final


In [11]:
g=Workbook()
final_answer = g.create_sheet()
B_SIZE = 1 # batch_size
for count in range(0,48):
  # 타이머 시작
  start = time.time()
  # 월별 거래내역, 12개월의 데이터. 반복문의 시작을 여기로 하면 될 듯.
  stock_trade_month = [g[1] for g in grouper[count].groupby(['기준년월'])]
  print(len(stock_trade_month))
  # 매수고객순으로 정렬
  for i in range(len(stock_trade_month)):
      stock_trade_month[i] = stock_trade_month[i].sort_values(by = ['매수고객수'], ascending = False)

  # 차례대로 선택된 그룹이 7~12월동안 각 월마다 제일 많이, 두번째로, 세번째로 많이 매수한 종목들
  first_buy_stock = [] 
  second_buy_stock = []
  third_buy_stock = []

  for i in range(len(stock_trade_month)-6):
      first_buy_stock.append(stock_trade_month[i+6].iloc[0]['종목번호']) 
      try:
        second_buy_stock.append(stock_trade_month[i+6].iloc[1]['종목번호'])
      except:
        pass
      try:
        third_buy_stock.append(stock_trade_month[i+6].iloc[2]['종목번호'])
      except:
        pass
      
      
  # 6개월별 데이터를 모아놓아야함
  six_months_data = []
  for i in range(len(stock_trade_month)-6):
      for j in range(6):
          six_months_data = six_months_data + stock_trade_month[i+j].loc[:,['종목번호']].values.tolist()
  six_months_stock_code = []
  for i in range(len(six_months_data)):
      six_months_stock_code.append(six_months_data[i][0])
  six_months_stock_encode = encoding_code(six_months_stock_code)

  # 6개월 데이터 합쳐서 데이터 프레임으로 표기
  final_df = DataFrame()
  for i in range(len(stock_trade_month)-6):
      for j in range(6):
          final_df = pd.concat([final_df,stock_trade_month[i+j]])
          
  final_df.drop(['Unnamed: 0','기준년월','그룹내고객수','그룹번호'], axis = 1)

  # 매수 매도 여부 encoding
  mesu_medo_mapping = {"Y": 1, "N": 0}
  final_df['그룹내_매수여부'] = final_df['그룹내_매수여부'].map(mesu_medo_mapping)
  final_df['그룹내_매도여부'] = final_df['그룹내_매도여부'].map(mesu_medo_mapping)

  # 매수매도 중앙값 처리
  final_df['중앙값차'] = 0
  final_df['중앙값차'] = round(((final_df['매도가격_중앙값'] - final_df['매수가격_중앙값'])/final_df['매수가격_중앙값'])*100,2)
  final_df['중앙값차'] = np.where(final_df['매도가격_중앙값']==-1, 30.00 , final_df['중앙값차'])
  final_df['중앙값차'] = np.where(final_df['매수가격_중앙값']==-1, -30.00 , final_df['중앙값차'])

  final_df= final_df.drop(['매도가격_중앙값'], axis = 1)
  final_df= final_df.drop(['매수가격_중앙값'], axis = 1)

  # multi class mixture embedding 
  for i in range(len(final_df)):
      six_months_stock_encode[i].append(final_df['그룹내_매수여부'].values[i])
      six_months_stock_encode[i].append(final_df['그룹내_매도여부'].values[i])
      six_months_stock_encode[i].append(final_df['매수고객수'].values[i])
      six_months_stock_encode[i].append(final_df['매도고객수'].values[i])
      six_months_stock_encode[i].append(final_df['평균매수수량'].values[i])
      six_months_stock_encode[i].append(final_df['평균매도수량'].values[i])
      six_months_stock_encode[i].append(final_df['중앙값차'].values[i])
      
  # target encoding
  target_first = []
  for i in range(len(stock_trade_month)-6):
      iter_num = 0
      for j in range(6):
          iter_num = iter_num + len(stock_trade_month[i+j])
      for k in range(iter_num):
          target_first.append(first_buy_stock[i])
  target_first = encoding_code(target_first)


  target_second = []
  for i in range(len(stock_trade_month)-6):
      iter_num = 0
      for j in range(6):
          iter_num = iter_num + len(stock_trade_month[i+j])
      for k in range(iter_num):
          try:
            target_second.append(second_buy_stock[i])
          except:
            target_second.append(first_buy_stock[i])    
  target_second = encoding_code(target_second)


  target_third= []
  for i in range(len(stock_trade_month)-6):
      iter_num = 0
      for j in range(6):
          iter_num = iter_num + len(stock_trade_month[i+j])
      for k in range(iter_num):
          try:
            target_third.append(third_buy_stock[i])
          except:
            target_third.append(first_buy_stock[i])
  target_third = encoding_code(target_third)

  # softmax Deep learning
  model_first = Sequential()
  model_second = Sequential()
  model_third = Sequential()


  model_first.add(Dense(1066, input_dim=1073, activation='softmax'))
  model_second.add(Dense(1066, input_dim=1073, activation='softmax'))
  model_third.add(Dense(1066, input_dim=1073, activation='softmax'))

  # learning rate = 0.1
  sgd=optimizers.SGD(lr=0.01)

  # 옵티마이저는 adam
  # loss function 은 categorical_crossentropy
  model_first.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  model_second.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  model_third.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


  

  history=model_first.fit(six_months_stock_encode,target_first, batch_size=B_SIZE, epochs=30)
  history=model_second.fit(six_months_stock_encode,target_second, batch_size=B_SIZE, epochs=30)
  history=model_third.fit(six_months_stock_encode,target_third, batch_size=B_SIZE, epochs=30)

  # Predict 
  predict_X = stock_trade_month[len(stock_trade_month)-1].loc[:,['종목번호']].values.tolist() 
  predict_X_code = []
  for i in range(len(predict_X)):
      predict_X_code.append(predict_X[i][0])
  predict_X_encode = encoding_code(predict_X_code)

  predict_df = stock_trade_month[len(stock_trade_month)-1]
  predict_df.drop(['Unnamed: 0','기준년월','그룹내고객수','그룹번호'], axis = 1)

  # 매수 매도 여부 encoding
  mesu_medo_mapping = {"Y": 1, "N": 0}
  predict_df['그룹내_매수여부'] = predict_df['그룹내_매수여부'].map(mesu_medo_mapping)
  predict_df['그룹내_매도여부'] = predict_df['그룹내_매도여부'].map(mesu_medo_mapping)

  # 매수매도 중앙값 처리
  predict_df['중앙값차'] = 0
  predict_df['중앙값차'] = round(((predict_df['매도가격_중앙값'] - predict_df['매수가격_중앙값'])/predict_df['매수가격_중앙값'])*100,2)
  predict_df['중앙값차'] = np.where(predict_df['매도가격_중앙값']==-1, 30.00 , predict_df['중앙값차'])
  predict_df['중앙값차'] = np.where(predict_df['매수가격_중앙값']==-1, -30.00 , predict_df['중앙값차'])

  predict_df= predict_df.drop(['매도가격_중앙값'], axis = 1)
  predict_df= predict_df.drop(['매수가격_중앙값'], axis = 1)

  # multi class mixture embedding 
  for i in range(len(predict_df)):
      predict_X_encode[i].append(predict_df['그룹내_매수여부'].values[i])
      predict_X_encode[i].append(predict_df['그룹내_매도여부'].values[i])
      predict_X_encode[i].append(predict_df['매수고객수'].values[i])
      predict_X_encode[i].append(predict_df['매도고객수'].values[i])
      predict_X_encode[i].append(predict_df['평균매수수량'].values[i])
      predict_X_encode[i].append(predict_df['평균매도수량'].values[i])
      predict_X_encode[i].append(predict_df['중앙값차'].values[i]) 

  pre_ans = []
  first_predict = all_stock.iloc[int(model_first.predict_classes([predict_X_encode[0]]))]['종목번호']
  pre_ans.append(first_predict)

  second_predict = all_stock.iloc[int(model_second.predict_classes([predict_X_encode[0]]))]['종목번호']
  if(first_predict == second_predict):
    second_predict =  all_stock.iloc[int(model_second.predict_classes([predict_X_encode[1]]))]['종목번호']
  pre_ans.append(second_predict)
  
  third_predict = all_stock.iloc[int(model_third.predict_classes([predict_X_encode[0]]))]['종목번호']
  if(third_predict == first_predict or third_predict == second_predict):
        third_predict = all_stock.iloc[int(model_third.predict_classes([predict_X_encode[1]]))]['종목번호']
        if(third_predict == first_predict or third_predict == second_predict):
            third_predict = all_stock.iloc[int(model_third.predict_classes([predict_X_encode[2]]))]['종목번호']
  pre_ans.append(third_predict)
  pre_ans.sort()
  final_answer.append(pre_ans)
  g.save(f_dir + '/answer.xlsx')
  print("--------{}그룹 ---------".format(count))
  print("time : ", time.time() - start) # 현재시각 - 시작시간
  

    



#주어진 X와 y데이터에 대해서 오차를 최소화하는 작업을 200번 시도합니다.

12
Epoch 1/30


InternalError:  Blas GEMM launch failed : a.shape=(1, 1073), b.shape=(1073, 1066), m=1, n=1066, k=1073
	 [[node sequential/dense/MatMul (defined at &lt;ipython-input-11-9b269a81c1c4&gt;:131) ]] [Op:__inference_train_function_640]

Function call stack:
train_function
